# Set up

## Model setup

In [ ]:
import subprocess

from sklearn.gaussian_process import GaussianProcessRegressor as GPR
from sklearn.gaussian_process import kernels
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.externals import joblib

import matplotlib.cm as cm
import matplotlib.pyplot as plt

from scipy.linalg import lapack
from scipy import stats
import emcee
import numpy as np

import os
import pickle
from pathlib import Path

import src.reader as Reader

## Step 1: prepare input pickle file

### Load stuff from text files

In [ ]:
# Read data files
RawData1M   = Reader.ReadData('input/MATTER/Data_PHENIX_AuAu200_RAACharged_0to10_2013.dat')
RawData2M   = Reader.ReadData('input/MATTER/Data_PHENIX_AuAu200_RAACharged_40to50_2013.dat')
RawData3M   = Reader.ReadData('input/MATTER/Data_ATLAS_PbPb2760_RAACharged_0to5_2015.dat')
RawData4M   = Reader.ReadData('input/MATTER/Data_ATLAS_PbPb2760_RAACharged_30to40_2015.dat')
RawData5M   = Reader.ReadData('input/MATTER/Data_CMS_PbPb5020_RAACharged_0to10_2017.dat')
RawData6M   = Reader.ReadData('input/MATTER/Data_CMS_PbPb5020_RAACharged_30to50_2017.dat')
RawData1MT  = Reader.ReadData('input/MATTERTruncated/Data_PHENIX_AuAu200_RAACharged_0to10_2013.dat')
RawData2MT  = Reader.ReadData('input/MATTERTruncated/Data_PHENIX_AuAu200_RAACharged_40to50_2013.dat')
RawData3MT  = Reader.ReadData('input/MATTERTruncated/Data_ATLAS_PbPb2760_RAACharged_0to5_2015.dat')
RawData4MT  = Reader.ReadData('input/MATTERTruncated/Data_ATLAS_PbPb2760_RAACharged_30to40_2015.dat')
RawData5MT  = Reader.ReadData('input/MATTERTruncated/Data_CMS_PbPb5020_RAACharged_0to10_2017.dat')
RawData6MT  = Reader.ReadData('input/MATTERTruncated/Data_CMS_PbPb5020_RAACharged_30to50_2017.dat')
RawData1L   = Reader.ReadData('input/LBTJake/Data_PHENIX_AuAu200_RAACharged_0to10_2013.dat')
RawData2L   = Reader.ReadData('input/LBTJake/Data_PHENIX_AuAu200_RAACharged_40to50_2013.dat')
RawData3L   = Reader.ReadData('input/LBTJake/Data_ATLAS_PbPb2760_RAACharged_0to5_2015.dat')
RawData4L   = Reader.ReadData('input/LBTJake/Data_ATLAS_PbPb2760_RAACharged_30to40_2015.dat')
RawData5L   = Reader.ReadData('input/LBTJake/Data_CMS_PbPb5020_RAACharged_0to10_2017.dat')
RawData6L   = Reader.ReadData('input/LBTJake/Data_CMS_PbPb5020_RAACharged_30to50_2017.dat')
RawData1ML1 = Reader.ReadData('input/MATTERLBT1/Data_PHENIX_AuAu200_RAACharged_0to10_2013.dat')
RawData2ML1 = Reader.ReadData('input/MATTERLBT1/Data_PHENIX_AuAu200_RAACharged_40to50_2013.dat')
RawData3ML1 = Reader.ReadData('input/MATTERLBT1/Data_ATLAS_PbPb2760_RAACharged_0to5_2015.dat')
RawData4ML1 = Reader.ReadData('input/MATTERLBT1/Data_ATLAS_PbPb2760_RAACharged_30to40_2015.dat')
RawData5ML1 = Reader.ReadData('input/MATTERLBT1/Data_CMS_PbPb5020_RAACharged_0to10_2017.dat')
RawData6ML1 = Reader.ReadData('input/MATTERLBT1/Data_CMS_PbPb5020_RAACharged_30to50_2017.dat')
RawData1ML2 = Reader.ReadData('input/MATTERLBT2/Data_PHENIX_AuAu200_RAACharged_0to10_2013.dat')
RawData2ML2 = Reader.ReadData('input/MATTERLBT2/Data_PHENIX_AuAu200_RAACharged_40to50_2013.dat')
RawData3ML2 = Reader.ReadData('input/MATTERLBT2/Data_ATLAS_PbPb2760_RAACharged_0to5_2015.dat')
RawData4ML2 = Reader.ReadData('input/MATTERLBT2/Data_ATLAS_PbPb2760_RAACharged_30to40_2015.dat')
RawData5ML2 = Reader.ReadData('input/MATTERLBT2/Data_CMS_PbPb5020_RAACharged_0to10_2017.dat')
RawData6ML2 = Reader.ReadData('input/MATTERLBT2/Data_CMS_PbPb5020_RAACharged_30to50_2017.dat')

# Read covariance
RawCov11L = Reader.ReadCovariance('input/LBT/Covariance_PHENIX_AuAu200_RAACharged_0to10_2013_PHENIX_AuAu200_RAACharged_0to10_2013_Jake.dat')
RawCov22L = Reader.ReadCovariance('input/LBT/Covariance_PHENIX_AuAu200_RAACharged_40to50_2013_PHENIX_AuAu200_RAACharged_40to50_2013_Jake.dat')
RawCov33L = Reader.ReadCovariance('input/LBT/Covariance_ATLAS_PbPb2760_RAACharged_0to5_2015_ATLAS_PbPb2760_RAACharged_0to5_2015_Jake.dat')
RawCov44L = Reader.ReadCovariance('input/LBT/Covariance_ATLAS_PbPb2760_RAACharged_30to40_2015_ATLAS_PbPb2760_RAACharged_30to40_2015_Jake.dat')
RawCov55L = Reader.ReadCovariance('input/LBT/Covariance_CMS_PbPb5020_RAACharged_0to10_2017_CMS_PbPb5020_RAACharged_0to10_2017_Jake.dat')
RawCov66L = Reader.ReadCovariance('input/LBT/Covariance_CMS_PbPb5020_RAACharged_30to50_2017_CMS_PbPb5020_RAACharged_30to50_2017_Jake.dat')

RawCov11E = Reader.ReadCovariance('input/Example/Covariance_PHENIX_AuAu200_RAACharged_0to10_2013_PHENIX_AuAu200_RAACharged_0to10_2013_SmallL.dat')
RawCov22E = Reader.ReadCovariance('input/Example/Covariance_PHENIX_AuAu200_RAACharged_40to50_2013_PHENIX_AuAu200_RAACharged_40to50_2013_SmallL.dat')
RawCov33E = Reader.ReadCovariance('input/Example/Covariance_ATLAS_PbPb2760_RAACharged_0to5_2015_ATLAS_PbPb2760_RAACharged_0to5_2015_SmallL.dat')
RawCov44E = Reader.ReadCovariance('input/Example/Covariance_ATLAS_PbPb2760_RAACharged_30to40_2015_ATLAS_PbPb2760_RAACharged_30to40_2015_SmallL.dat')
RawCov55E = Reader.ReadCovariance('input/Example/Covariance_CMS_PbPb5020_RAACharged_0to10_2017_CMS_PbPb5020_RAACharged_0to10_2017_SmallL.dat')
RawCov66E = Reader.ReadCovariance('input/Example/Covariance_CMS_PbPb5020_RAACharged_30to50_2017_CMS_PbPb5020_RAACharged_30to50_2017_SmallL.dat')

# Read design points
RawDesignM   = Reader.ReadDesign('input/MATTER/Design.dat')
RawDesignMT  = Reader.ReadDesign('input/MATTERTruncated/Design.dat')
RawDesignL   = Reader.ReadDesign('input/LBTJake/Design.dat')
RawDesignML1 = Reader.ReadDesign('input/MATTERLBT1/Design.dat')
RawDesignML2 = Reader.ReadDesign('input/MATTERLBT2/Design.dat')

# Read model prediction
RawPrediction1M   = Reader.ReadPrediction('input/MATTER/Prediction_PHENIX_AuAu200_RAACharged_0to10_2013.dat')
RawPrediction2M   = Reader.ReadPrediction('input/MATTER/Prediction_PHENIX_AuAu200_RAACharged_40to50_2013.dat')
RawPrediction3M   = Reader.ReadPrediction('input/MATTER/Prediction_ATLAS_PbPb2760_RAACharged_0to5_2015.dat')
RawPrediction4M   = Reader.ReadPrediction('input/MATTER/Prediction_ATLAS_PbPb2760_RAACharged_30to40_2015.dat')
RawPrediction5M   = Reader.ReadPrediction('input/MATTER/Prediction_CMS_PbPb5020_RAACharged_0to10_2017.dat')
RawPrediction6M   = Reader.ReadPrediction('input/MATTER/Prediction_CMS_PbPb5020_RAACharged_30to50_2017.dat')
RawPrediction1MT  = Reader.ReadPrediction('input/MATTERTruncated/Prediction_PHENIX_AuAu200_RAACharged_0to10_2013.dat')
RawPrediction2MT  = Reader.ReadPrediction('input/MATTERTruncated/Prediction_PHENIX_AuAu200_RAACharged_40to50_2013.dat')
RawPrediction3MT  = Reader.ReadPrediction('input/MATTERTruncated/Prediction_ATLAS_PbPb2760_RAACharged_0to5_2015.dat')
RawPrediction4MT  = Reader.ReadPrediction('input/MATTERTruncated/Prediction_ATLAS_PbPb2760_RAACharged_30to40_2015.dat')
RawPrediction5MT  = Reader.ReadPrediction('input/MATTERTruncated/Prediction_CMS_PbPb5020_RAACharged_0to10_2017.dat')
RawPrediction6MT  = Reader.ReadPrediction('input/MATTERTruncated/Prediction_CMS_PbPb5020_RAACharged_30to50_2017.dat')
RawPrediction1L   = Reader.ReadPrediction('input/LBTJake/Prediction_PHENIX_AuAu200_RAACharged_0to10_2013.dat')
RawPrediction2L   = Reader.ReadPrediction('input/LBTJake/Prediction_PHENIX_AuAu200_RAACharged_40to50_2013.dat')
RawPrediction3L   = Reader.ReadPrediction('input/LBTJake/Prediction_ATLAS_PbPb2760_RAACharged_0to5_2015.dat')
RawPrediction4L   = Reader.ReadPrediction('input/LBTJake/Prediction_ATLAS_PbPb2760_RAACharged_30to40_2015.dat')
RawPrediction5L   = Reader.ReadPrediction('input/LBTJake/Prediction_CMS_PbPb5020_RAACharged_0to10_2017.dat')
RawPrediction6L   = Reader.ReadPrediction('input/LBTJake/Prediction_CMS_PbPb5020_RAACharged_30to50_2017.dat')
RawPrediction1ML1 = Reader.ReadPrediction('input/MATTERLBT1/Prediction_PHENIX_AuAu200_RAACharged_0to10_2013.dat')
RawPrediction2ML1 = Reader.ReadPrediction('input/MATTERLBT1/Prediction_PHENIX_AuAu200_RAACharged_40to50_2013.dat')
RawPrediction3ML1 = Reader.ReadPrediction('input/MATTERLBT1/Prediction_ATLAS_PbPb2760_RAACharged_0to5_2015.dat')
RawPrediction4ML1 = Reader.ReadPrediction('input/MATTERLBT1/Prediction_ATLAS_PbPb2760_RAACharged_30to40_2015.dat')
RawPrediction5ML1 = Reader.ReadPrediction('input/MATTERLBT1/Prediction_CMS_PbPb5020_RAACharged_0to10_2017.dat')
RawPrediction6ML1 = Reader.ReadPrediction('input/MATTERLBT1/Prediction_CMS_PbPb5020_RAACharged_30to50_2017.dat')
RawPrediction1ML2 = Reader.ReadPrediction('input/MATTERLBT2/Prediction_PHENIX_AuAu200_RAACharged_0to10_2013.dat')
RawPrediction2ML2 = Reader.ReadPrediction('input/MATTERLBT2/Prediction_PHENIX_AuAu200_RAACharged_40to50_2013.dat')
RawPrediction3ML2 = Reader.ReadPrediction('input/MATTERLBT2/Prediction_ATLAS_PbPb2760_RAACharged_0to5_2015.dat')
RawPrediction4ML2 = Reader.ReadPrediction('input/MATTERLBT2/Prediction_ATLAS_PbPb2760_RAACharged_30to40_2015.dat')
RawPrediction5ML2 = Reader.ReadPrediction('input/MATTERLBT2/Prediction_CMS_PbPb5020_RAACharged_0to10_2017.dat')
RawPrediction6ML2 = Reader.ReadPrediction('input/MATTERLBT2/Prediction_CMS_PbPb5020_RAACharged_30to50_2017.dat')

### Run this block for RHIC + LHC (MATTER), high PT points only

In [ ]:
# Initialize empty dictionary
AllData = {}

# Basic information
AllData["systems"] = ["AuAu200", "PbPb2760", "PbPb5020"]
AllData["keys"] = RawDesignMT["Parameter"]
AllData["labels"] = RawDesignMT["Parameter"]
AllData["ranges"] = [(0, 2), (0, 20), (0, 2), (0, 20)]
AllData["observables"] = [('R_AA', ['C0', 'C1'])]

# Data points
Data = {"AuAu200": {"R_AA": {"C0": RawData1MT["Data"], "C1": RawData2MT["Data"]}},
    "PbPb2760": {"R_AA": {"C0": RawData3MT["Data"], "C1": RawData4MT["Data"]}},
    "PbPb5020": {"R_AA": {"C0": RawData5MT["Data"], "C1": RawData6MT["Data"]}}}

# Model predictions
Prediction = {"AuAu200": {"R_AA": {"C0": {"Y": RawPrediction1MT["Prediction"], "x": RawData1MT["Data"]['x']},
                                   "C1": {"Y": RawPrediction2MT["Prediction"], "x": RawData2MT["Data"]['x']}}},
             "PbPb2760": {"R_AA": {"C0": {"Y": RawPrediction3MT["Prediction"], "x": RawData3MT["Data"]['x']},
                                   "C1": {"Y": RawPrediction4MT["Prediction"], "x": RawData4MT["Data"]['x']}}},
             "PbPb5020": {"R_AA": {"C0": {"Y": RawPrediction5MT["Prediction"], "x": RawData5MT["Data"]['x']},
                                   "C1": {"Y": RawPrediction6MT["Prediction"], "x": RawData6MT["Data"]['x']}}}}

# Covariance matrices - the indices are [system][measurement1][measurement2], each one is a block of matrix
Covariance = Reader.InitializeCovariance(Data)
Covariance["AuAu200"][("R_AA", "C0")][("R_AA", "C0")] = Reader.EstimateCovariance(RawData1MT, RawData1MT, SysLength = {"default": 0.2})
Covariance["AuAu200"][("R_AA", "C1")][("R_AA", "C1")] = Reader.EstimateCovariance(RawData2MT, RawData2MT, SysLength = {"default": 0.2})
Covariance["PbPb2760"][("R_AA", "C0")][("R_AA", "C0")] = Reader.EstimateCovariance(RawData3MT, RawData3MT, SysLength = {"default": 0.2})
Covariance["PbPb2760"][("R_AA", "C1")][("R_AA", "C1")] = Reader.EstimateCovariance(RawData4MT, RawData4MT, SysLength = {"default": 0.2})
Covariance["PbPb5020"][("R_AA", "C0")][("R_AA", "C0")] = Reader.EstimateCovariance(RawData5MT, RawData5MT, SysLength = {"default": 0.2})
Covariance["PbPb5020"][("R_AA", "C1")][("R_AA", "C1")] = Reader.EstimateCovariance(RawData6MT, RawData6MT, SysLength = {"default": 0.2})

# This is how we can add off-diagonal matrices
# Covariance["PbPb5020"][("R_AA", "C0")][("R_AA", "C1")] = Reader.EstimateCovariance(RawData5, RawData6, SysLength = {"default": 100}, SysStrength = {"default": 0.1})
# Covariance["PbPb5020"][("R_AA", "C1")][("R_AA", "C0")] = Reader.EstimateCovariance(RawData6, RawData5, SysLength = {"default": 100}, SysStrength = {"default": 0.1})

# This is how we can supply external pre-generated matrices
# Covariance["AuAu200"][("R_AA", "C0")][("R_AA", "C0")] = RawCov11E["Matrix"]
# Covariance["AuAu200"][("R_AA", "C1")][("R_AA", "C1")] = RawCov22E["Matrix"]
# Covariance["PbPb2760"][("R_AA", "C0")][("R_AA", "C0")] = RawCov33E["Matrix"]
# Covariance["PbPb2760"][("R_AA", "C1")][("R_AA", "C1")] = RawCov44E["Matrix"]
# Covariance["PbPb5020"][("R_AA", "C0")][("R_AA", "C0")] = RawCov55E["Matrix"]
# Covariance["PbPb5020"][("R_AA", "C1")][("R_AA", "C1")] = RawCov66E["Matrix"]


# Assign data to the dictionary
AllData["design"] = RawDesignMT["Design"]
AllData["model"] = Prediction
AllData["data"] = Data
AllData["cov"] = Covariance

# Save to the desired pickle file
with open('input/default.p', 'wb') as handle:
    pickle.dump(AllData, handle, protocol = pickle.HIGHEST_PROTOCOL)

### Run this block for RHIC + LHC (MATTER), all PT points

In [ ]:
# Initialize empty dictionary
AllData = {}

# Basic information
AllData["systems"] = ["AuAu200", "PbPb2760", "PbPb5020"]
AllData["keys"] = RawDesignM["Parameter"]
AllData["labels"] = RawDesignM["Parameter"]
AllData["ranges"] = [(0, 2), (0, 20), (0, 2), (0, 20)]
AllData["observables"] = [('R_AA', ['C0', 'C1'])]

# Data points
Data = {"AuAu200": {"R_AA": {"C0": RawData1M["Data"], "C1": RawData2M["Data"]}},
    "PbPb2760": {"R_AA": {"C0": RawData3M["Data"], "C1": RawData4M["Data"]}},
    "PbPb5020": {"R_AA": {"C0": RawData5M["Data"], "C1": RawData6M["Data"]}}}

# Model predictions
Prediction = {"AuAu200": {"R_AA": {"C0": {"Y": RawPrediction1M["Prediction"], "x": RawData1M["Data"]['x']},
                                   "C1": {"Y": RawPrediction2M["Prediction"], "x": RawData2M["Data"]['x']}}},
             "PbPb2760": {"R_AA": {"C0": {"Y": RawPrediction3M["Prediction"], "x": RawData3M["Data"]['x']},
                                   "C1": {"Y": RawPrediction4M["Prediction"], "x": RawData4M["Data"]['x']}}},
             "PbPb5020": {"R_AA": {"C0": {"Y": RawPrediction5M["Prediction"], "x": RawData5M["Data"]['x']},
                                   "C1": {"Y": RawPrediction6M["Prediction"], "x": RawData6M["Data"]['x']}}}}

# Covariance matrices - the indices are [system][measurement1][measurement2], each one is a block of matrix
Covariance = Reader.InitializeCovariance(Data)
Covariance["AuAu200"][("R_AA", "C0")][("R_AA", "C0")] = Reader.EstimateCovariance(RawData1M, RawData1M, SysLength = {"default": 0.2})
Covariance["AuAu200"][("R_AA", "C1")][("R_AA", "C1")] = Reader.EstimateCovariance(RawData2M, RawData2M, SysLength = {"default": 0.2})
Covariance["PbPb2760"][("R_AA", "C0")][("R_AA", "C0")] = Reader.EstimateCovariance(RawData3M, RawData3M, SysLength = {"default": 0.2})
Covariance["PbPb2760"][("R_AA", "C1")][("R_AA", "C1")] = Reader.EstimateCovariance(RawData4M, RawData4M, SysLength = {"default": 0.2})
Covariance["PbPb5020"][("R_AA", "C0")][("R_AA", "C0")] = Reader.EstimateCovariance(RawData5M, RawData5M, SysLength = {"default": 0.2})
Covariance["PbPb5020"][("R_AA", "C1")][("R_AA", "C1")] = Reader.EstimateCovariance(RawData6M, RawData6M, SysLength = {"default": 0.2})

# This is how we can add off-diagonal matrices
# Covariance["PbPb5020"][("R_AA", "C0")][("R_AA", "C1")] = Reader.EstimateCovariance(RawData5, RawData6, SysLength = {"default": 100}, SysStrength = {"default": 0.1})
# Covariance["PbPb5020"][("R_AA", "C1")][("R_AA", "C0")] = Reader.EstimateCovariance(RawData6, RawData5, SysLength = {"default": 100}, SysStrength = {"default": 0.1})

# This is how we can supply external pre-generated matrices
# Covariance["AuAu200"][("R_AA", "C0")][("R_AA", "C0")] = RawCov11E["Matrix"]
# Covariance["AuAu200"][("R_AA", "C1")][("R_AA", "C1")] = RawCov22E["Matrix"]
# Covariance["PbPb2760"][("R_AA", "C0")][("R_AA", "C0")] = RawCov33E["Matrix"]
# Covariance["PbPb2760"][("R_AA", "C1")][("R_AA", "C1")] = RawCov44E["Matrix"]
# Covariance["PbPb5020"][("R_AA", "C0")][("R_AA", "C0")] = RawCov55E["Matrix"]
# Covariance["PbPb5020"][("R_AA", "C1")][("R_AA", "C1")] = RawCov66E["Matrix"]


# Assign data to the dictionary
AllData["design"] = RawDesignM["Design"]
AllData["model"] = Prediction
AllData["data"] = Data
AllData["cov"] = Covariance

# Save to the desired pickle file
with open('input/default.p', 'wb') as handle:
    pickle.dump(AllData, handle, protocol = pickle.HIGHEST_PROTOCOL)

### Run this block for RHIC + LHC (MATTER) -- only one group

In [ ]:
# Initialize empty dictionary
AllData = {}

# Basic information
AllData["systems"] = ["HiHi100"]
AllData["keys"] = RawDesignM["Parameter"]
AllData["labels"] = RawDesignM["Parameter"]
AllData["ranges"] = [(0, 2), (0, 20), (0, 2), (0, 20)]
AllData["observables"] = [('R_AA', ['C0', 'C1', 'C2', 'C3', 'C4', 'C5'])]

# Data points
Data = {"HiHi100": {"R_AA": {"C0": RawData1M["Data"], "C1": RawData2M["Data"],
    "C2": RawData3M["Data"], "C3": RawData4M["Data"],
    "C4": RawData5M["Data"], "C5": RawData6M["Data"]}}}

# Model predictions
Prediction = {"HiHi100": {"R_AA": {"C0": {"Y": RawPrediction1M["Prediction"], "x": RawData1M["Data"]['x']},
                                     "C1": {"Y": RawPrediction2M["Prediction"], "x": RawData2M["Data"]['x']},
                                     "C2": {"Y": RawPrediction3M["Prediction"], "x": RawData3M["Data"]['x']},
                                     "C3": {"Y": RawPrediction4M["Prediction"], "x": RawData4M["Data"]['x']},
                                     "C4": {"Y": RawPrediction5M["Prediction"], "x": RawData5M["Data"]['x']},
                                     "C5": {"Y": RawPrediction6M["Prediction"], "x": RawData6M["Data"]['x']}}}}
                                    

# Covariance matrices - the indices are [system][measurement1][measurement2], each one is a block of matrix
Covariance = Reader.InitializeCovariance(Data)
Covariance["HiHi100"][("R_AA", "C0")][("R_AA", "C0")] = Reader.EstimateCovariance(RawData1M, RawData1M, SysLength = {"default": 0.2})
Covariance["HiHi100"][("R_AA", "C1")][("R_AA", "C1")] = Reader.EstimateCovariance(RawData2M, RawData2M, SysLength = {"default": 0.2})
Covariance["HiHi100"][("R_AA", "C2")][("R_AA", "C2")] = Reader.EstimateCovariance(RawData3M, RawData3M, SysLength = {"default": 0.2})
Covariance["HiHi100"][("R_AA", "C3")][("R_AA", "C3")] = Reader.EstimateCovariance(RawData4M, RawData4M, SysLength = {"default": 0.2})
Covariance["HiHi100"][("R_AA", "C4")][("R_AA", "C4")] = Reader.EstimateCovariance(RawData5M, RawData5M, SysLength = {"default": 0.2})
Covariance["HiHi100"][("R_AA", "C5")][("R_AA", "C5")] = Reader.EstimateCovariance(RawData6M, RawData6M, SysLength = {"default": 0.2})

# This is how we can add off-diagonal matrices
# Covariance["PbPb5020"][("R_AA", "C0")][("R_AA", "C1")] = Reader.EstimateCovariance(RawData5, RawData6, SysLength = {"default": 100}, SysStrength = {"default": 0.1})
# Covariance["PbPb5020"][("R_AA", "C1")][("R_AA", "C0")] = Reader.EstimateCovariance(RawData6, RawData5, SysLength = {"default": 100}, SysStrength = {"default": 0.1})

# This is how we can supply external pre-generated matrices
# Covariance["AuAu200"][("R_AA", "C0")][("R_AA", "C0")] = RawCov1["Matrix"]
# Covariance["AuAu200"][("R_AA", "C0")][("R_AA", "C0")] = RawCov11E["Matrix"]
# Covariance["AuAu200"][("R_AA", "C1")][("R_AA", "C1")] = RawCov22E["Matrix"]
# Covariance["PbPb2760"][("R_AA", "C0")][("R_AA", "C0")] = RawCov33E["Matrix"]
# Covariance["PbPb2760"][("R_AA", "C1")][("R_AA", "C1")] = RawCov44E["Matrix"]
# Covariance["PbPb5020"][("R_AA", "C0")][("R_AA", "C0")] = RawCov55E["Matrix"]
# Covariance["PbPb5020"][("R_AA", "C1")][("R_AA", "C1")] = RawCov66E["Matrix"]


# Assign data to the dictionary
AllData["design"] = RawDesignM["Design"]
AllData["model"] = Prediction
AllData["data"] = Data
AllData["cov"] = Covariance

# Save to the desired pickle file
with open('input/default.p', 'wb') as handle:
    pickle.dump(AllData, handle, protocol = pickle.HIGHEST_PROTOCOL)

### Run this block for RHIC + LHC (LBT)

In [ ]:
# Initialize empty dictionary
AllData = {}

# Basic information
AllData["systems"] = ["AuAu200", "PbPb2760", "PbPb5020"]
AllData["keys"] = RawDesignL["Parameter"]
AllData["labels"] = RawDesignL["Parameter"]
AllData["ranges"] = [(0.01, 2), (0.01, 20), (0.01, 2), (0.01, 20)]
AllData["observables"] = [('R_AA', ['C0', 'C1'])]

# Data points
Data = {"AuAu200": {"R_AA": {"C0": RawData1L["Data"], "C1": RawData2L["Data"]}},
       "PbPb2760": {"R_AA": {"C0": RawData3L["Data"], "C1": RawData4L["Data"]}},
       "PbPb5020": {"R_AA": {"C0": RawData5L["Data"], "C1": RawData6L["Data"]}}}

# Model predictions
Prediction = {"AuAu200": {"R_AA": {"C0": {"Y": RawPrediction1L["Prediction"], "x": RawData1L["Data"]['x']},
                                   "C1": {"Y": RawPrediction2L["Prediction"], "x": RawData2L["Data"]['x']}}},
             "PbPb2760": {"R_AA": {"C0": {"Y": RawPrediction3L["Prediction"], "x": RawData3L["Data"]['x']},
                                   "C1": {"Y": RawPrediction4L["Prediction"], "x": RawData4L["Data"]['x']}}},
             "PbPb5020": {"R_AA": {"C0": {"Y": RawPrediction5L["Prediction"], "x": RawData5L["Data"]['x']},
                                   "C1": {"Y": RawPrediction6L["Prediction"], "x": RawData6L["Data"]['x']}}}}

# Covariance matrices - the indices are [system][measurement1][measurement2], each one is a block of matrix
Covariance = Reader.InitializeCovariance(Data)
Covariance["AuAu200"][("R_AA", "C0")][("R_AA", "C0")]  = Reader.EstimateCovariance(RawData1L, RawData1L, SysLength = {"default": 0.2})
Covariance["AuAu200"][("R_AA", "C1")][("R_AA", "C1")]  = Reader.EstimateCovariance(RawData2L, RawData2L, SysLength = {"default": 0.2})
Covariance["PbPb2760"][("R_AA", "C0")][("R_AA", "C0")] = Reader.EstimateCovariance(RawData3L, RawData3L, SysLength = {"default": 0.2})
Covariance["PbPb2760"][("R_AA", "C1")][("R_AA", "C1")] = Reader.EstimateCovariance(RawData4L, RawData4L, SysLength = {"default": 0.2})
Covariance["PbPb5020"][("R_AA", "C0")][("R_AA", "C0")] = Reader.EstimateCovariance(RawData5L, RawData5L, SysLength = {"default": 0.2})
Covariance["PbPb5020"][("R_AA", "C1")][("R_AA", "C1")] = Reader.EstimateCovariance(RawData6L, RawData6L, SysLength = {"default": 0.2})

# This is how we can add off-diagonal matrices
# Covariance["PbPb5020"][("R_AA", "C0")][("R_AA", "C1")] = Reader.EstimateCovariance(RawData5, RawData6, SysLength = {"default": 100}, SysStrength = {"default": 0.1})
# Covariance["PbPb5020"][("R_AA", "C1")][("R_AA", "C0")] = Reader.EstimateCovariance(RawData6, RawData5, SysLength = {"default": 100}, SysStrength = {"default": 0.1})

# This is how we can supply external pre-generated matrices
# Covariance["AuAu200"][("R_AA", "C0")][("R_AA", "C0")] = RawCov11L["Matrix"]
# Covariance["AuAu200"][("R_AA", "C1")][("R_AA", "C1")] = RawCov22L["Matrix"]
# Covariance["PbPb2760"][("R_AA", "C0")][("R_AA", "C0")] = RawCov33L["Matrix"]
# Covariance["PbPb2760"][("R_AA", "C1")][("R_AA", "C1")] = RawCov44L["Matrix"]
# Covariance["PbPb5020"][("R_AA", "C0")][("R_AA", "C0")] = RawCov55L["Matrix"]
# Covariance["PbPb5020"][("R_AA", "C1")][("R_AA", "C1")] = RawCov66L["Matrix"]

# Assign data to the dictionary
AllData["design"] = RawDesignL["Design"]
AllData["model"] = Prediction
AllData["data"] = Data
AllData["cov"] = Covariance

# Save to the desired pickle file
with open('input/default.p', 'wb') as handle:
    pickle.dump(AllData, handle, protocol = pickle.HIGHEST_PROTOCOL)

### Run this block for RHIC + LHC (MATTER+LBT1)

In [ ]:
# Initialize empty dictionary
AllData = {}

# Basic information
AllData["systems"] = ["AuAu200", "PbPb2760", "PbPb5020"]
AllData["keys"] = RawDesignML1["Parameter"]
AllData["labels"] = RawDesignML1["Parameter"]
AllData["ranges"] = [(0, 1.5), (0, 1.0), (0, 20), (0, 20), (1, 4)]
AllData["observables"] = [('R_AA', ['C0', 'C1'])]

# Data points
Data = {"AuAu200": {"R_AA": {"C0": RawData1ML1["Data"], "C1": RawData2ML1["Data"]}},
    "PbPb2760": {"R_AA": {"C0": RawData3ML1["Data"], "C1": RawData4ML1["Data"]}},
    "PbPb5020": {"R_AA": {"C0": RawData5ML1["Data"], "C1": RawData6ML1["Data"]}}}

# Model predictions
Prediction = {"AuAu200": {"R_AA": {"C0": {"Y": RawPrediction1ML1["Prediction"], "x": RawData1ML1["Data"]['x']},
                                   "C1": {"Y": RawPrediction2ML1["Prediction"], "x": RawData2ML1["Data"]['x']}}},
             "PbPb2760": {"R_AA": {"C0": {"Y": RawPrediction3ML1["Prediction"], "x": RawData3ML1["Data"]['x']},
                                   "C1": {"Y": RawPrediction4ML1["Prediction"], "x": RawData4ML1["Data"]['x']}}},
             "PbPb5020": {"R_AA": {"C0": {"Y": RawPrediction5ML1["Prediction"], "x": RawData5ML1["Data"]['x']},
                                   "C1": {"Y": RawPrediction6ML1["Prediction"], "x": RawData6ML1["Data"]['x']}}}}

# Covariance matrices - the indices are [system][measurement1][measurement2], each one is a block of matrix
Covariance = Reader.InitializeCovariance(Data)
Covariance["AuAu200"][("R_AA", "C0")][("R_AA", "C0")] = Reader.EstimateCovariance(RawData1ML1, RawData1ML1, SysLength = {"default": 0.2})
Covariance["AuAu200"][("R_AA", "C1")][("R_AA", "C1")] = Reader.EstimateCovariance(RawData2ML1, RawData2ML1, SysLength = {"default": 0.2})
Covariance["PbPb2760"][("R_AA", "C0")][("R_AA", "C0")] = Reader.EstimateCovariance(RawData3ML1, RawData3ML1, SysLength = {"default": 0.2})
Covariance["PbPb2760"][("R_AA", "C1")][("R_AA", "C1")] = Reader.EstimateCovariance(RawData4ML1, RawData4ML1, SysLength = {"default": 0.2})
Covariance["PbPb5020"][("R_AA", "C0")][("R_AA", "C0")] = Reader.EstimateCovariance(RawData5ML1, RawData5ML1, SysLength = {"default": 0.2})
Covariance["PbPb5020"][("R_AA", "C1")][("R_AA", "C1")] = Reader.EstimateCovariance(RawData6ML1, RawData6ML1, SysLength = {"default": 0.2})

# This is how we can add off-diagonal matrices
# Covariance["PbPb5020"][("R_AA", "C0")][("R_AA", "C1")] = Reader.EstimateCovariance(RawData5, RawData6, SysLength = {"default": 100}, SysStrength = {"default": 0.1})
# Covariance["PbPb5020"][("R_AA", "C1")][("R_AA", "C0")] = Reader.EstimateCovariance(RawData6, RawData5, SysLength = {"default": 100}, SysStrength = {"default": 0.1})

# This is how we can supply external pre-generated matrices
# Covariance["AuAu200"][("R_AA", "C0")][("R_AA", "C0")] = RawCov1["Matrix"]


# Assign data to the dictionary
AllData["design"] = RawDesignML1["Design"]
AllData["model"] = Prediction
AllData["data"] = Data
AllData["cov"] = Covariance

# Save to the desired pickle file
with open('input/default.p', 'wb') as handle:
    pickle.dump(AllData, handle, protocol = pickle.HIGHEST_PROTOCOL)

### Run this block for RHIC + LHC (MATTER+LBT2)

In [ ]:
# Initialize empty dictionary
AllData = {}

# Basic information
AllData["systems"] = ["AuAu200", "PbPb2760", "PbPb5020"]
AllData["keys"] = RawDesignML2["Parameter"]
AllData["labels"] = RawDesignML2["Parameter"]
AllData["ranges"] = [(0, 1.5), (0, 1.0), (0, 20), (1, 4)]
AllData["observables"] = [('R_AA', ['C0', 'C1'])]

# Data points
Data = {"AuAu200": {"R_AA": {"C0": RawData1ML2["Data"], "C1": RawData2ML2["Data"]}},
    "PbPb2760": {"R_AA": {"C0": RawData3ML2["Data"], "C1": RawData4ML2["Data"]}},
    "PbPb5020": {"R_AA": {"C0": RawData5ML2["Data"], "C1": RawData6ML2["Data"]}}}

# Model predictions
Prediction = {"AuAu200": {"R_AA": {"C0": {"Y": RawPrediction1ML2["Prediction"], "x": RawData1ML2["Data"]['x']},
                                   "C1": {"Y": RawPrediction2ML2["Prediction"], "x": RawData2ML2["Data"]['x']}}},
             "PbPb2760": {"R_AA": {"C0": {"Y": RawPrediction3ML2["Prediction"], "x": RawData3ML2["Data"]['x']},
                                   "C1": {"Y": RawPrediction4ML2["Prediction"], "x": RawData4ML2["Data"]['x']}}},
             "PbPb5020": {"R_AA": {"C0": {"Y": RawPrediction5ML2["Prediction"], "x": RawData5ML2["Data"]['x']},
                                   "C1": {"Y": RawPrediction6ML2["Prediction"], "x": RawData6ML2["Data"]['x']}}}}

# Covariance matrices - the indices are [system][measurement1][measurement2], each one is a block of matrix
Covariance = Reader.InitializeCovariance(Data)
Covariance["AuAu200"][("R_AA", "C0")][("R_AA", "C0")] = Reader.EstimateCovariance(RawData1ML2, RawData1ML2, SysLength = {"default": 0.2})
Covariance["AuAu200"][("R_AA", "C1")][("R_AA", "C1")] = Reader.EstimateCovariance(RawData2ML2, RawData2ML2, SysLength = {"default": 0.2})
Covariance["PbPb2760"][("R_AA", "C0")][("R_AA", "C0")] = Reader.EstimateCovariance(RawData3ML2, RawData3ML2, SysLength = {"default": 0.2})
Covariance["PbPb2760"][("R_AA", "C1")][("R_AA", "C1")] = Reader.EstimateCovariance(RawData4ML2, RawData4ML2, SysLength = {"default": 0.2})
Covariance["PbPb5020"][("R_AA", "C0")][("R_AA", "C0")] = Reader.EstimateCovariance(RawData5ML2, RawData5ML2, SysLength = {"default": 0.2})
Covariance["PbPb5020"][("R_AA", "C1")][("R_AA", "C1")] = Reader.EstimateCovariance(RawData6ML2, RawData6ML2, SysLength = {"default": 0.2})

# This is how we can add off-diagonal matrices
# Covariance["PbPb5020"][("R_AA", "C0")][("R_AA", "C1")] = Reader.EstimateCovariance(RawData5, RawData6, SysLength = {"default": 100}, SysStrength = {"default": 0.1})
# Covariance["PbPb5020"][("R_AA", "C1")][("R_AA", "C0")] = Reader.EstimateCovariance(RawData6, RawData5, SysLength = {"default": 100}, SysStrength = {"default": 0.1})

# This is how we can supply external pre-generated matrices
# Covariance["AuAu200"][("R_AA", "C0")][("R_AA", "C0")] = RawCov1["Matrix"]


# Assign data to the dictionary
AllData["design"] = RawDesignML2["Design"]
AllData["model"] = Prediction
AllData["data"] = Data
AllData["cov"] = Covariance

# Save to the desired pickle file
with open('input/default.p', 'wb') as handle:
    pickle.dump(AllData, handle, protocol = pickle.HIGHEST_PROTOCOL)

In [ ]:
print(AllData["design"].shape)

print(AllData["model"]["AuAu200"]["R_AA"]["C0"]["Y"].shape)
print(AllData["model"]["AuAu200"]["R_AA"]["C1"]["Y"].shape)
print(AllData["model"]["PbPb2760"]["R_AA"]["C0"]["Y"].shape)
print(AllData["model"]["PbPb2760"]["R_AA"]["C1"]["Y"].shape)
print(AllData["model"]["PbPb5020"]["R_AA"]["C0"]["Y"].shape)
print(AllData["model"]["PbPb5020"]["R_AA"]["C1"]["Y"].shape)

In [ ]:
print("AuAu200, C0")
print(Covariance["AuAu200"][("R_AA", "C0")][("R_AA", "C0")])
print("AuAu200, C1")
print(Covariance["AuAu200"][("R_AA", "C1")][("R_AA", "C1")])
print("PbPb2760, C0")
print(Covariance["PbPb2760"][("R_AA", "C0")][("R_AA", "C0")])
print("PbPb2760, C1")
print(Covariance["PbPb2760"][("R_AA", "C1")][("R_AA", "C1")])

In [ ]:
SystemCount = len(AllData["systems"])

figure, axes = plt.subplots(figsize = (15, 5 * SystemCount), ncols = 2, nrows = SystemCount)

axes[0][0].imshow((Covariance["AuAu200"][("R_AA", "C0")][("R_AA", "C0")]))
axes[0][1].imshow((Covariance["AuAu200"][("R_AA", "C1")][("R_AA", "C1")]))
axes[1][0].imshow((Covariance["PbPb2760"][("R_AA", "C0")][("R_AA", "C0")]))
axes[1][1].imshow((Covariance["PbPb2760"][("R_AA", "C1")][("R_AA", "C1")]))
axes[2][0].imshow((Covariance["PbPb5020"][("R_AA", "C0")][("R_AA", "C0")]))
axes[2][1].imshow((Covariance["PbPb5020"][("R_AA", "C1")][("R_AA", "C1")]))

figure.tight_layout()

In [ ]:
SystemCount = 3

figure, axes = plt.subplots(figsize = (15, 5 * SystemCount), ncols = 2, nrows = SystemCount)

axes[0][0].imshow((Covariance["HiHi100"][("R_AA", "C0")][("R_AA", "C0")]))
axes[0][1].imshow((Covariance["HiHi100"][("R_AA", "C1")][("R_AA", "C1")]))
axes[1][0].imshow((Covariance["HiHi100"][("R_AA", "C2")][("R_AA", "C2")]))
axes[1][1].imshow((Covariance["HiHi100"][("R_AA", "C3")][("R_AA", "C3")]))
axes[2][0].imshow((Covariance["HiHi100"][("R_AA", "C4")][("R_AA", "C4")]))
axes[2][1].imshow((Covariance["HiHi100"][("R_AA", "C5")][("R_AA", "C5")]))

figure.tight_layout()

### Optional: clean past files

In [ ]:
# Clean past MCMC samples
if os.path.exists('cache/mcmc_chain.hdf'):
    os.remove("cache/mcmc_chain.hdf")

# Clean past emulator
for system in AllData["systems"]:
    if os.path.exists('cache/emulator/' + system + ".pkl"):
        os.remove('cache/emulator/' + system + ".pkl")

## Step 2: run emulator

In [ ]:
! python3 -m src.emulator --retrain --npc 2 --nrestarts 50

In [ ]:
from src import lazydict, emulator
EmulatorAuAu200 = emulator.Emulator.from_cache('AuAu200')
EmulatorPbPb2760 = emulator.Emulator.from_cache('PbPb2760')
EmulatorPbPb5020 = emulator.Emulator.from_cache('PbPb5020')

## Step 3: MCMC sampling

In [ ]:
if os.path.exists('cache/mcmc_chain.hdf'):
    os.remove("cache/mcmc_chain.hdf")
! python3 -m src.mcmc --nwalkers 500 --nburnsteps 2000 3000

## Step 4: Analyze posterior samples

In [ ]:
import src
src.Initialize()
from src import mcmc
chain = mcmc.Chain()
MCMCSamples = chain.load()

In [ ]:
print(MCMCSamples)
print(MCMCSamples.shape)

## Step 5: adding all sorts of plots

In [ ]:
import src
src.Initialize()
from src import mcmc
chain = mcmc.Chain()
MCMCSamples = chain.load()

In [ ]:
with chain.dataset() as d:
    W = d.shape[0]     # number of walkers
    S = d.shape[1]     # number of steps
    N = d.shape[2]     # number of paramters
    T = int(S / 200)   # "thinning"
    A = 20 / W
    figure, axes = plt.subplots(figsize = (15, 2 * N), ncols = 1, nrows = N)
    for i, ax in enumerate(axes):
        for j in range(0, W):
            ax.plot(range(0, S, T), d[j, ::T, i], alpha = A)
    plt.tight_layout(True)
    plt.savefig('plots/MCMCSamples.pdf', dpi = 192)

In [ ]:
NDimension = len(AllData["labels"])
Ranges = np.array(AllData["ranges"]).T
figure, axes = plt.subplots(figsize = (3 * NDimension, 3 * NDimension), ncols = NDimension, nrows = NDimension)
Names = AllData["labels"]
for i, row in enumerate(axes):
    for j, ax in enumerate(row):
        if i==j:
            ax.hist(MCMCSamples[:,i], bins=50,
                    range=Ranges[:,i], histtype='step', color='green')
            ax.set_xlabel(Names[i])
            ax.set_xlim(*Ranges[:,j])
        if i>j:
            ax.hist2d(MCMCSamples[:, j], MCMCSamples[:, i], 
                      bins=50, range=[Ranges[:,j], Ranges[:,i]], 
                      cmap='Greens')
            ax.set_xlabel(Names[j])
            ax.set_ylabel(Names[i])
            ax.set_xlim(*Ranges[:,j])
            ax.set_ylim(*Ranges[:,i])
        if i<j:
            ax.axis('off')
plt.tight_layout(True)
plt.savefig('plots/Correlation.pdf', dpi = 192)
# figure

In [ ]:
Examples = MCMCSamples[ np.random.choice(range(len(MCMCSamples)), 2500), :]

TempPrediction = {"AuAu200": EmulatorAuAu200.predict(Examples),
                 "PbPb2760": EmulatorPbPb2760.predict(Examples),
                 "PbPb5020": EmulatorPbPb5020.predict(Examples)}

SystemCount = len(AllData["systems"])

figure, axes = plt.subplots(figsize = (15, 5 * SystemCount), ncols = 2, nrows = SystemCount)

for s1 in range(0, SystemCount):
    for s2 in range(0, 2):
        axes[s1][s2].set_xlabel(r"$p_{T}$")
        axes[s1][s2].set_ylabel(r"$R_{AA}$")
        
        S1 = AllData["systems"][s1]
        O  = AllData["observables"][0][0]
        S2 = AllData["observables"][0][1][s2]

        DX = AllData["data"][S1][O][S2]['x']
        DY = AllData["data"][S1][O][S2]['y']
        DE = np.sqrt(AllData["data"][S1][O][S2]['yerr']['stat'][:,0]**2 + AllData["data"][S1][O][S2]['yerr']['sys'][:,0]**2)
                
        for i, y in enumerate(TempPrediction[S1][O][S2]):
            axes[s1][s2].plot(DX, y, 'b-', alpha=0.005, label="Posterior" if i==0 else '')
        axes[s1][s2].errorbar(DX, DY, yerr = DE, fmt='ro', label="Measurements")

plt.tight_layout(True)
figure.savefig('plots/ObservablePosterior.pdf', dpi = 192)
# figure

In [ ]:
Examples = MCMCSamples[ np.random.choice(range(len(MCMCSamples)), 2500), :]

TempPrediction = {"AuAu200": EmulatorAuAu200.predict(Examples),
                 "PbPb2760": EmulatorPbPb2760.predict(Examples),
                 "PbPb5020": EmulatorPbPb5020.predict(Examples)}

SystemCount = len(AllData["systems"])

figure, axes = plt.subplots(figsize = (15, 5 * SystemCount), ncols = 2, nrows = SystemCount)

for s1 in range(0, SystemCount):
    for s2 in range(0, 2):
        axes[s1][s2].set_xlabel(r"$p_{T}$")
        axes[s1][s2].set_ylabel("$R_{AA}$: Pull = (prediction - data) / error")
        
        S1 = AllData["systems"][s1]
        O  = AllData["observables"][0][0]
        S2 = AllData["observables"][0][1][s2]

        DX = AllData["data"][S1][O][S2]['x']
        DY = AllData["data"][S1][O][S2]['y']
        DE = np.sqrt(AllData["data"][S1][O][S2]['yerr']['stat'][:,0]**2 + AllData["data"][S1][O][S2]['yerr']['sys'][:,0]**2)
                
        for i, y in enumerate(TempPrediction[S1][O][S2]):
            axes[s1][s2].plot(DX, (y - DY) / DE, 'b-', alpha=0.005, label="Posterior" if i==0 else '')
        # axes[s1][s2].errorbar(DX, DY, yerr = DE, fmt='ro', label="Measurements")

plt.tight_layout(True)
figure.savefig('plots/ObservablePosteriorPull.pdf', dpi = 192)
figure.savefig('plots/ObservablePosteriorPull.png', dpi = 192)
# figure

In [ ]:
TransformedSamples = np.copy(MCMCSamples)
TransformedSamples[:,0] = MCMCSamples[:,0] * MCMCSamples[:,1]
TransformedSamples[:,1] = MCMCSamples[:,0] - MCMCSamples[:,0] * MCMCSamples[:,1]
TransformedSamples[:,2] = MCMCSamples[:,2]
TransformedSamples[:,3] = MCMCSamples[:,3]
# TransformedSamples[:,4] = MCMCSamples[:,4]

In [ ]:
! python3 -m src.plots posterior gp diag_emu

In [ ]:
# close all plots to save memory
plt.close('all')

In [ ]:
# ! python3 -m src.simpleplots

## Step 6: debug the hell out of it

In [ ]:
# First let's sketch out the likelihood landscape as seen by the mcmc sampler
A = 0.45
B = 2
C = 0.35
D = 15

# A = 0.1
# B = 1
# C = 0.4
# D = 1

figure, axis = plt.subplots(figsize = (10, 10), ncols = 2, nrows = 2)
model_cov_modifier = 0
model_sys_error = False

N = 200
axis[0][0].scatter(np.linspace(0, 2, N), chain.log_posterior([[A, B, C, D] for A in np.linspace(0, 2, N)], model_cov_modifier = model_cov_modifier, model_sys_error = model_sys_error))
axis[0][0].set_xlabel("A")
axis[0][0].set_ylabel("log(Posterior)")
axis[0][1].scatter(np.linspace(0, 20, N), chain.log_posterior([[A, B, C, D] for B in np.linspace(0, 20, N)], model_cov_modifier = model_cov_modifier, model_sys_error = model_sys_error))
axis[0][1].set_xlabel("B")
axis[0][1].set_ylabel("log(Posterior)")
axis[1][0].scatter(np.linspace(0, 2, N), chain.log_posterior([[A, B, C, D] for C in np.linspace(0, 2, N)], model_cov_modifier = model_cov_modifier, model_sys_error = model_sys_error))
axis[1][0].set_xlabel("C")
axis[1][0].set_ylabel("log(Posterior)")
axis[1][1].scatter(np.linspace(0, 20, N), chain.log_posterior([[A, B, C, D] for D in np.linspace(0, 20, N)], model_cov_modifier = model_cov_modifier, model_sys_error = model_sys_error))
axis[1][1].set_xlabel("D")
axis[1][1].set_ylabel("log(Posterior)")

In [ ]:
A = 0.45
B = 2.00
C = 0.35
D = 15

# A = 0.1
# B = 1
# C = 0.4
# D = 1

N = 50
model_cov_modifier = 0
model_sys_error = False

TestSamples = np.array([[[A, B, C, D] for C in np.linspace(0.02, 1.99, N)] for D in np.linspace(0.02, 19.99, N)])
TestLikelihood = chain.log_posterior(TestSamples.reshape(-1, 4), model_cov_modifier = model_cov_modifier, model_sys_error = model_sys_error).reshape(N, N)
figure, axis = plt.subplots(figsize = (10, 10))
im = axis.imshow(TestLikelihood, interpolation="none", origin='lower', extent = (0, 2, 0, 20), aspect='auto', cmap='nipy_spectral')
axis.set_xlabel("C")
axis.set_ylabel("D")
figure.colorbar(im)

In [ ]:
A = 0.45
B = 2.00
C = 0.35
D = 15

# A = 0.1
# B = 1
# C = 0.4
# D = 1

N = 50
model_cov_modifier = 1
model_sys_error = False

TestSamples = np.array([[[A, B, C, D] for C in np.linspace(0.02, 0.39, N)] for D in np.linspace(0.02, 19.99, N)])
TestLikelihood = chain.log_posterior(TestSamples.reshape(-1, 4), model_cov_modifier = model_cov_modifier, model_sys_error = model_sys_error).reshape(N, N)
figure, axis = plt.subplots(figsize = (10, 10))
im = axis.imshow(TestLikelihood, interpolation="none", origin='lower', extent = (0, 0.4, 0, 20), aspect='auto', cmap='nipy_spectral')
axis.set_xlabel("C")
axis.set_ylabel("D")
figure.colorbar(im)

In [ ]:
A = 0.45
B = 2.00
C = 0.35
D = 15

# A = 0.1
# B = 1
# C = 0.4
# D = 1

N = 50
TestSamples = np.array([[[A / 2, B / 20, C, D] for C in np.linspace(0.01, 0.20, N)] for D in np.linspace(0.01, 0.99, N)]).reshape(-1, 4)
Distance = np.full((N, N), 9999).reshape(-1)
for Point in AllData["design"]:
    VarPoint = [Point[0] / 2, Point[1] / 20, Point[2] / 2, Point[3] / 20]
    DistanceToPoint = np.array([np.linalg.norm(VarPoint - X) for X in TestSamples])
    Distance = np.minimum(Distance, DistanceToPoint)

figure, axis = plt.subplots(figsize = (10, 10))
im = axis.imshow(Distance.reshape(N, N), interpolation="none", origin='lower', extent = (0, 0.2, 0, 1), aspect='auto')
axis.set_xlabel("Scaled C")
axis.set_ylabel("Scaled D")
figure.colorbar(im)

In [ ]:
A = 0.45
B = 2.00
C = 0.35
D = 15
N = 50

# A = 0.1
# B = 1
# C = 0.4
# D = 1

TestSamples = np.array([[[A, B, C, D] for C in np.linspace(0.02, 1.99, N)] for D in np.linspace(0.02, 19.99, N)])
Prediction = chain._predict(TestSamples.reshape(-1, 4))

figure, axis = plt.subplots(figsize = (10, 10))

System = "AuAu200"
Observable = "R_AA"
Centrality = "C1"
Index = 0

Diff = Prediction[System][Observable][Centrality][:,Index].reshape(N, N) - Data[System][Observable][Centrality]["y"][Index]

xv, yv = np.meshgrid(np.linspace(0.02, 1.99, N), np.linspace(0.02, 19.99, N), sparse=False, indexing='xy')

im = axis.imshow(Diff, interpolation="none", origin='lower', extent = (0, 2, 0, 20), aspect='auto')
axis.contour(xv, yv, Diff, levels = [0])
axis.set_xlabel("C")
axis.set_ylabel("D")
figure.colorbar(im)

In [ ]:
A = 0.45
B = 2.00
C = 0.35
D = 15
N = 50

# A = 0.1
# B = 1
# C = 0.4
# D = 1

TestSamples = np.array([[[A, B, C, D] for C in np.linspace(0.02, 0.49, N)] for D in np.linspace(0.02, 19.99, N)])
Prediction = chain._predict(TestSamples.reshape(-1, 4))

figure, axis = plt.subplots(figsize = (10, 15), ncols = 2, nrows = 3)

System = ["AuAu200", "PbPb2760", "PbPb5020"]
Observable = "R_AA"
Centrality = ["C0", "C1"]

xv, yv = np.meshgrid(np.linspace(0.02, 0.49, N), np.linspace(0.02, 19.99, N), sparse=False, indexing='xy')

Colors = ['b', 'g', 'r', 'y', 'm', 'c', 'k', 'b', 'g', 'r', 'y', 'm', 'c', 'k']

for S in range(0, 3):
    for C in range(0, 2):
        for Index in range(np.array(Prediction[System[S]][Observable][Centrality[C]]).shape[1]):
            Diff = Prediction[System[S]][Observable][Centrality[C]][:,Index].reshape(N, N) - Data[System[S]][Observable][Centrality[C]]["y"][Index]
            CS = axis[S][C].contour(xv, yv, Diff, levels = [0], colors = Colors[Index])
            axis[S][C].clabel(CS, CS.levels, inline=True, fontsize=10, fmt=(str(Index) + ' (%d)'))

            axis[S][C].set_xlabel("C")
            axis[S][C].set_ylabel("D")


In [ ]:
Examples = [[0.45, 2.00, 0.05, 0.01],
            [0.45, 2.00, 0.35, 15],
            [0.45, 2.00, 0.10, 9],
            [0.55, 0.01, 0.01, 0.01],
            [0.55, 0.01, 0.01, 12.00]]

TempPrediction = {"AuAu200": EmulatorAuAu200.predict(Examples),
                 "PbPb2760": EmulatorPbPb2760.predict(Examples),
                 "PbPb5020": EmulatorPbPb5020.predict(Examples)}

SystemCount = len(AllData["systems"])

figure, axes = plt.subplots(figsize = (15, 5 * SystemCount), ncols = 2, nrows = SystemCount)

Colors = ['b', 'g', 'r', 'y', 'c', 'm', 'k', 'b', 'g', 'r', 'y', 'm', 'c', 'k']

for s1 in range(0, SystemCount):
    for s2 in range(0, 2):
        axes[s1][s2].set_xlabel(r"$p_{T}$")
        axes[s1][s2].set_ylabel(r"$R_{AA}$")
        
        S1 = AllData["systems"][s1]
        O  = AllData["observables"][0][0]
        S2 = AllData["observables"][0][1][s2]

        DX = AllData["data"][S1][O][S2]['x']
        DY = AllData["data"][S1][O][S2]['y']
        DE = np.sqrt(AllData["data"][S1][O][S2]['yerr']['stat'][:,0]**2 + AllData["data"][S1][O][S2]['yerr']['sys'][:,0]**2)
                
        for i, y in enumerate(TempPrediction[S1][O][S2]):
            axes[s1][s2].plot(DX, y, Colors[i] + '-', alpha=1, linewidth=3.0, label="Posterior" if i==0 else '')
        axes[s1][s2].errorbar(DX, DY, yerr = DE, fmt='ro', label="Measurements")

plt.tight_layout(True)
figure.savefig('plots/ExamplePoints.pdf', dpi = 192)
figure.savefig('plots/ExamplePoints.png', dpi = 192)
# figure

print(chain.log_posterior(Examples))

In [ ]:
Examples = np.array([[0.45, 2.00, 0.05, 0.01],
                     [0.45, 2.00, 0.35, 15],
                     [0.45, 2.00, 0.10, 9],
                     [0.55, 0.01, 0.01, 0.01],
                     [0.55, 0.01, 0.01, 12.00]])

TempPrediction = {"AuAu200": EmulatorAuAu200.predict(Examples),
                 "PbPb2760": EmulatorPbPb2760.predict(Examples),
                 "PbPb5020": EmulatorPbPb5020.predict(Examples)}

SystemCount = len(AllData["systems"])

figure, axes = plt.subplots(figsize = (15, 5 * SystemCount), ncols = 2, nrows = SystemCount)

Colors = ['b', 'g', 'r', 'y', 'c', 'm', 'k', 'b', 'g', 'r', 'y', 'm', 'c', 'k']

AbsPull = np.zeros(Examples.shape[0])
Chi2 = np.zeros(Examples.shape[0])

for s1 in range(0, SystemCount):
    for s2 in range(0, 2):
        axes[s1][s2].set_xlabel(r"$p_{T}$")
        axes[s1][s2].set_ylabel("$R_{AA}$: Pull = (prediction - data) / error")
        
        S1 = AllData["systems"][s1]
        O  = AllData["observables"][0][0]
        S2 = AllData["observables"][0][1][s2]

        DX = AllData["data"][S1][O][S2]['x']
        DY = AllData["data"][S1][O][S2]['y']
        DE = np.sqrt(AllData["data"][S1][O][S2]['yerr']['stat'][:,0]**2 + AllData["data"][S1][O][S2]['yerr']['sys'][:,0]**2)
                
        for i, y in enumerate(TempPrediction[S1][O][S2]):
            axes[s1][s2].plot(DX, (y - DY) / DE, Colors[i] + 'o-', alpha=1, linewidth=3.0, label="Posterior" if i==0 else '')
            AbsPull[i] = AbsPull[i] + np.sum(np.abs((y - DY) / DE))
            Chi2[i] = Chi2[i] + np.sum(np.abs((y - DY) / DE)**2)
        # axes[s1][s2].errorbar(DX, DY, yerr = DE, fmt='ro', label="Measurements")

plt.tight_layout(True)
figure.savefig('plots/ExamplePointsPull.pdf', dpi = 192)
figure.savefig('plots/ExamplePointsPull.png', dpi = 192)
# figure

print(AbsPull)
print(Chi2)

In [ ]:
# close all plots to save memory
plt.close('all')

In [ ]:
Data["PbPb2760"]["R_AA"]["C0"]

In [ ]:
NDimension = len(AllData["labels"])
Ranges = np.array(AllData["ranges"]).T
figure, axes = plt.subplots(figsize = (15, 15), ncols = NDimension, nrows = NDimension)
Names = [r"$A$", r"$C$", r"$B$", r"$D$", r"$Q$", r"$P_6$"]
for i, row in enumerate(axes):
    for j, ax in enumerate(row):
        if i==j:
            ax.hist(TransformedSamples[:,i], bins=50,
                    range=Ranges[:,i], histtype='step')
            ax.set_xlabel(Names[i])
            ax.set_xlim(*Ranges[:,j])
        if i>j:
            ax.hist2d(TransformedSamples[:, j], TransformedSamples[:, i], 
                      bins=50, range=[Ranges[:,j], Ranges[:,i]], 
                      cmap='Blues')
            ax.set_xlabel(Names[j])
            ax.set_ylabel(Names[i])
            ax.set_xlim(*Ranges[:,j])
            ax.set_ylim(*Ranges[:,i])
        if i<j:
            ax.axis('off')
plt.tight_layout(True)
plt.savefig('plots/TransformedCorrelation.pdf', dpi = 192)
# figure